In [99]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


Load Data

In [100]:
db = pd.read_csv('data.csv')
db.head

<bound method NDFrame.head of                                                message  fingers tail   species
0                                   pluvia arbor aquos        4   no    Aquari
1                            cosmix xeno nebuz odbitaz        5  yes   Zorblax
2                   solarix glixx novum galaxum quasar        5  yes   Zorblax
3             arbor insectus pesros ekos dootix nimbus        2  yes   Florian
4                    mermax drakos lorix epikoz deftax        4   no    Faerix
..                                                 ...      ...  ...       ...
495  empathix sadix disgux dredax pridius afgstix e...        2   no  Emotivor
496                      quasar ustron nebulax meteorn        4   no   Quixnar
497                 astron xeno ceaestar astron kometa        6  yes   Zorblax
498  sporzom nimbus terram terranix aviana ekos nimbub        2  yes   Florian
499     blastix titanos relikum drakos gryphox sirenix        4  yes   Mythron

[500 rows x 4 columns

In [101]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  500 non-null    object
 1   fingers  500 non-null    int64 
 2   tail     500 non-null    object
 3   species  500 non-null    object
dtypes: int64(1), object(3)
memory usage: 15.8+ KB


In [102]:
db.duplicated().sum()

np.int64(0)

No null as well as duplicate data

In [103]:
db['tail'].value_counts()

tail
no     252
yes    248
Name: count, dtype: int64

In [104]:
for x in range(len(db['tail'])):
    if(db['tail'][x]=="yes"):
        db.loc[x, "tail"] = 1
    else:
        db.loc[x, "tail"] = 0
db

,message,fingers,tail,species
0,pluvia arbor aquos,4,0,Aquari
1,cosmix xeno nebuz odbitaz,5,1,Zorblax
2,solarix glixx novum galaxum quasar,5,1,Zorblax
3,arbor insectus pesros ekos dootix nimbus,2,1,Florian
4,mermax drakos lorix epikoz deftax,4,0,Faerix
...,...,...,...,...
495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor
496,quasar ustron nebulax meteorn,4,0,Quixnar
497,astron xeno ceaestar astron kometa,6,1,Zorblax
498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian


In [106]:
# tfidf_vectorizer = TfidfVectorizer()
# X_tfidf = tfidf_vectorizer.fit_transform(db['message'])
# print(X_tfidf.toarray())
# print(tfidf_vectorizer.get_feature_names_out())


db['message'].value_counts()

message
blastix titanos relikum drakos gryphox sirenix                 1
pluvia arbor aquos                                             1
cosmix xeno nebuz odbitaz                                      1
solarix glixx novum galaxum quasar                             1
mechan virtua quantaz codex tecfnos                            1
                                                              ..
dredax ragex empathix anxius                                   1
legax fenix herox sagaq                                        1
datax quantaz cryptoz cybron quantix nanozom cryptoz neuraz    1
synapz algorix cryptoz digitvon algorix quantix robonix        1
mermax drakos lorix epikoz deftax                              1
Name: count, Length: 500, dtype: int64